In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Colab 환경설정

!pip install gluonnlp pandas tqdm
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch==1.8.1

In [ ]:
# github에서 Kobert 파일 로드 및 Kobert 모델 불러오기

#kobert_tokenizer 폴더 다운받는 코드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

#https://github.com/SKTBrain/KoBERT 의 파일들을 다운받는 코드
!pip install 'git+https://git@github.com/SKTBrain/KoBERT.git@master'

from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
# 필요한 라이브러리 불러오기

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))
  
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

In [ ]:
# 데이터셋 확인
data = pd.read_csv('감정분류데이터셋.csv', encoding='cp949')

In [ ]:
# 7개의 감정 class를 0~6 숫자에 대응시켜 data_list에 담아준다
data.loc[(data['상황'] == "fear"), '상황'] = 0  #공포 => 0
data.loc[(data['상황'] == "surprise"), '상황'] = 1  #놀람 => 1
data.loc[(data['상황'] == "angry"), '상황'] = 2  #분노 => 2
data.loc[(data['상황'] == "sadness"), '상황'] = 3  #슬픔 => 3
data.loc[(data['상황'] == "neutral"), '상황'] = 4  #중립 => 4
data.loc[(data['상황'] == "happiness"), '상황'] = 5  #행복 => 5
data.loc[(data['상황'] == "disgust"), '상황'] = 6  #혐오 => 6


data_list = []
for ques, label in zip(data['발화문'], data['상황'])  :
    data = []   
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [ ]:
# 입력 데이터셋을 토큰화하기

# 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding padding 등을 해주는 코드이다.
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5  
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)
dataset_train, dataset_validation = train_test_split(dataset_train, test_size=0.2, shuffle=True, random_state=34)

# train 데이터와 test 데이터 갯수 각각 출력
print(len(data_list))
print(len(dataset_train))
print(len(dataset_validation))
print(len(dataset_test))

In [ ]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_valid = BERTDataset(dataset_validation,0, 1, tok, vocab,  max_len, True, False) 
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=5)

In [ ]:
# KoBERT 모델 구현하기

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
# train

In [ ]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

In [ ]:
## 학습 모델 저장
PATH = '/content/drive/MyDrive/colab/model_1000' # google 드라이브 연동 해야함.
torch.save(model, PATH + 'model_1000.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

In [ ]:
# 직접 만든 새로운 문장으로 테스트

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
        

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

**성능평가하기 - 필수 실행 셀들**

In [ ]:
#BERT 모델 불러오기
PATH = 'drive/MyDrive/colab/'
model = torch.load(PATH + 'model_1000model_1000.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load(PATH + 'model_1000model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

In [ ]:
# 결과 분석을 위한 confusion_matrix
# 참고 https://ai-creator.tistory.com/579
matrix = []

def init_matrix():
  global matrix
  matrix.clear()

  matrix = [[0 for col in range(8)] for row in range(8)]
  matrix[0][0] = " "
  for i in range(1,8) :
    matrix[i][0] = i-1
    matrix[0][i] = i-1



In [ ]:
# confusion_matrix 출력 함수 
def print_confusion_matrix():
  print("공포(0) 놀람(1) 분노(2) 슬픔(3) 중립(4) 행복(5) 혐오(6) ")
  print()
  for i in matrix :
    for j in i:
        print(j,end=" ")
    print()

In [ ]:
# 오차행렬, 정확도, 정밀도, 재현율을 한번에 출력하기 위한 함수 정의

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred, average= "macro")
    recall = recall_score(y_test, pred, average= "macro")
    print('Confusion Matrix')
    print(confusion)
    print('정확도:{}, 정밀도:{}, 재현율:{}'.format(accuracy, precision, recall))

In [ ]:
def predict2(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            return np.argmax(logits)


**선택 1) 학습과 다른 형식의 데이터셋으로 평가**

In [ ]:
test_data = pd.read_csv('한국어_단발성_대화_데이터셋.csv', encoding='UTF8')

# 7개의 감정 class를 0~6 숫자에 대응시켜 data_list에 담아준다
test_data.loc[(test_data['상황'] == "공포"), '상황'] = 0  #공포 => 0
test_data.loc[(test_data['상황'] == "놀람"), '상황'] = 1  #놀람 => 1
test_data.loc[(test_data['상황'] == "분노"), '상황'] = 2  #분노 => 2
test_data.loc[(test_data['상황'] == "슬픔"), '상황'] = 3  #슬픔 => 3
test_data.loc[(test_data['상황'] == "중립"), '상황'] = 4  #중립 => 4
test_data.loc[(test_data['상황'] == "행복"), '상황'] = 5  #행복 => 5
test_data.loc[(test_data['상황'] == "혐오"), '상황'] = 6  #혐오 => 6


test_data_list = []
for ques, label in zip(test_data['Sentence'], test_data['상황'])  :
    test_data = []   
    test_data.append(ques)
    test_data.append(label)

    test_data_list.append(test_data)

In [ ]:
# 감정 분류당 500개씩 추출
import random

x_test = []
x_true = []

for i in range(500):
  choice = random.choice(test_data_list[0:2000])
  x_test.append(choice)

for i in range(500):
  choice = random.choice(test_data_list[6000:10000])
  x_test.append(choice)

for i in range(500):
  choice = random.choice(test_data_list[12000:17000])
  x_test.append(choice)

for i in range(500):
  choice = random.choice(test_data_list[18000:22000])
  x_test.append(choice)

for i in range(500):
  choice = random.choice(test_data_list[23000:27000])
  x_test.append(random.choice(test_data_list[23000:27000]))

for i in range(500):
  choice = random.choice(test_data_list[28000:33000])
  x_test.append(choice)

for i in range(500):
  choice = random.choice(test_data_list[34000:38000])
  x_test.append(choice)

In [ ]:
cnt = 0
pred = []
x_true = []
init_matrix()

for sentence, emotion in x_test:
  cnt += 1
  if cnt % 100 == 0:
    print(cnt, end = " ")
  print()
    
  result = predict2(sentence)
  pred.append(result)
  x_true.append(emotion)
  matrix[emotion+1][result+1] = matrix[emotion+1][result+1] + 1

print_confusion_matrix()
get_clf_eval(x_true, pred)

**선택 2) 감성분류 데이터셋 20퍼센트를 학습에 사용하지 않고 남겨둔 뒤 성능평가에 활용**

In [ ]:
cnt = 0
pred = []
x_true = []

for sentence, emotion in dataset_test:
  cnt += 1
  if cnt % 100 == 0:
    print(cnt, end = " ")
    print()
    
  result = predict2(sentence)
  pred.append(result)
  x_true.append(int(emotion))
  matrix[int(emotion)+1][result+1] = matrix[int(emotion)+1][result+1] + 1

print_confusion_matrix()
get_clf_eval(x_true, pred)

**선택 3) 섞어서 쓰자**
250개를 한국어 단발성 데이터셋에서, 250개는 dataset_test에서 

In [ ]:
test_data = pd.read_csv('한국어_단발성_대화_데이터셋.csv', encoding='UTF8')

# 7개의 감정 class를 0~6 숫자에 대응시켜 data_list에 담아준다
test_data.loc[(test_data['상황'] == "공포"), '상황'] = 0  #공포 => 0
test_data.loc[(test_data['상황'] == "놀람"), '상황'] = 1  #놀람 => 1
test_data.loc[(test_data['상황'] == "분노"), '상황'] = 2  #분노 => 2
test_data.loc[(test_data['상황'] == "슬픔"), '상황'] = 3  #슬픔 => 3
test_data.loc[(test_data['상황'] == "중립"), '상황'] = 4  #중립 => 4
test_data.loc[(test_data['상황'] == "행복"), '상황'] = 5  #행복 => 5
test_data.loc[(test_data['상황'] == "혐오"), '상황'] = 6  #혐오 => 6


test_data_list = []
for ques, label in zip(test_data['Sentence'], test_data['상황'])  :
    test_data = []   
    test_data.append(ques)
    test_data.append(label)

    test_data_list.append(test_data)

In [ ]:
import random

x_test = []

# 한국어 단발성 데이터셋에서 250개
for j in range(250):
  choice = random.choice(test_data_list[0:2000])
  x_test.append(choice)

for i in range(250):
  choice = random.choice(test_data_list[6000:10000])
  x_test.append(choice)

for i in range(250):
  choice = random.choice(test_data_list[12000:17000])
  x_test.append(choice)

for i in range(250):
  choice = random.choice(test_data_list[18000:22000])
  x_test.append(choice)

for i in range(250):
  choice = random.choice(test_data_list[23000:27000])
  x_test.append(random.choice(test_data_list[23000:27000]))

for i in range(250):
  choice = random.choice(test_data_list[28000:33000])
  x_test.append(choice)

for i in range(250):
  choice = random.choice(test_data_list[34000:38000])
  x_test.append(choice)

# 감정분류 데이터셋에서 250*7개 추가
x_test.extend(dataset_test[0:250*7])


In [ ]:
cnt = 0
pred = []
x_true = []
init_matrix()

for sentence, emotion in x_test:
  cnt += 1
  if cnt % 100 == 0:
    print(cnt, end = " ")
  print()
    
  result = predict2(sentence)
  pred.append(result)
  x_true.append(int(emotion))
  matrix[int(emotion)+1][result+1] = matrix[int(emotion)+1][result+1] + 1

print_confusion_matrix()
print()
get_clf_eval(x_true, pred)